<a href="https://colab.research.google.com/github/Aissa-C/Anal-tica/blob/main/Reto_2_AD_Aissa_Ceballos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reto 2. Modelo de Árboles de Decisión**
0. Cargue de las librerías de referencia para el modelo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Cargue de los datos del archivo Travel Data

In [ ]:
nxl= '/content/drive/MyDrive/Analítica de Negocios/2. BD1_Travel_Data_int.xlsx'
XDB= pd.read_excel(nxl, sheet_name=0)
XDB= XDB.dropna()

XDB['College']= XDB['College'].map({'Yes':1, 'No':0})
XDB['CreditCard']= XDB['CreditCard'].map({'Yes':1, 'No':0})
print(XDB.head(100))

#Selección de variables
XD= np.array(XDB[['College', 'FoodSpend', 'Income', 'TravelSpend']])
print(XD)
yd= np.array(XDB['CreditCard'])
print(yd)

    College  CreditCard  FoodSpend  Income  TravelSpend
0         1           1    5472.43   49150       827.40
1         0           1    9130.73   47806       863.55
2         0           1    4450.67   46050      1180.05
3         0           1    5020.72   42600       755.70
4         0           1    3408.11   52092       851.40
..      ...         ...        ...     ...          ...
95        0           0    2019.98   65982      4338.75
96        0           0    7046.11   53986      1851.60
97        0           0    5180.13   53986      1646.70
98        0           0    3266.44   63962      1641.60
99        1           1    2082.23   49004      1564.95

[100 rows x 5 columns]
[[1.00000e+00 5.47243e+03 4.91500e+04 8.27400e+02]
 [0.00000e+00 9.13073e+03 4.78060e+04 8.63550e+02]
 [0.00000e+00 4.45067e+03 4.60500e+04 1.18005e+03]
 ...
 [0.00000e+00 3.58586e+03 4.22500e+04 2.25195e+03]
 [1.00000e+00 1.83395e+03 6.02480e+04 2.00025e+03]
 [0.00000e+00 6.20597e+03 5.85200e+04 3.6675

2. Creación del Modelo: Árbol de Decisión

In [ ]:
mar= DecisionTreeClassifier(criterion='gini', max_depth=4)
mar= mar.fit(XD, yd)
ydp= mar.predict(XD)

from sklearn.metrics import confusion_matrix
cm= confusion_matrix(yd, ydp)
print(cm)

#Evaluación del modelo
VN= cm[0,0]; FP= cm[0,1]; FN= cm[1,0]; VP= cm[1,1]

Ex= (VP+VN)/(VP+VN+FP+FN)  #Porcentaje de datos que pronosticó correctamente
print('La exactitud del modelo es: ', Ex)

Se= VP/(VP+FN)  #Porcentaje de datos positivos que pronosticó correctamente
print('La sensibilidad del modelo es: ', Se)

Es= VN/(VN+FP)  #Porcentaje de datos negativos que pronosticó correctamente
print('La especificidad del modelo es: ', Es)

[[334   9]
 [ 42 115]]
La exactitud del modelo es:  0.898
La sensibilidad del modelo es:  0.732484076433121
La especificidad del modelo es:  0.9737609329446064


3. Árbol de Decisión

In [ ]:
#Librerías para graficar el árbol
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

vs= ['College', 'FoodSpend', 'Income', 'TravelSpend']
dot_graph= export_graphviz(mar, feature_names=vs)
graph= graph_from_dot_data(dot_graph)
graph.write_png('arbol_reto.png')

True

4. Pronóstico de los nuevos clientes

In [ ]:
XDB1= pd.read_excel(nxl, sheet_name=1)
XDB1= XDB1.dropna()
XDB1['College']= XDB1['College'].map({'Yes':1, 'No':0})
print(XDB1)

XD1= np.array(XDB1[['College', 'FoodSpend', 'Income', 'TravelSpend']])
print(XD1)

#Pronosticamos la utilización
ydp1= mar.predict(XD1)
print(ydp1)
print("Las personas que usarán su tarjeta de crédito son:", np.sum(ydp1))
print("Las personas que NO usarán su tarjeta de crédito son:", len(ydp1)-np.sum(ydp1))

   College  FoodSpend  Income  TravelSpend
0        0    2892.90   65982       2151.6
1        1    6017.66   53986       2002.5
[[0.00000e+00 2.89290e+03 6.59820e+04 2.15160e+03]
 [1.00000e+00 6.01766e+03 5.39860e+04 2.00250e+03]]
[0 0]
Las personas que usarán su tarjeta de crédito son: 0
Las personas que NO usarán su tarjeta de crédito son: 2


**Análisis de Resultados**

De acuerdo con la base de datos suministrada y el nodo base del árbol de decisión, se puede observar un total de 500 datos, divididos en 343 usuarios de la tarjeta de crédito y un total de 157 no usuarios de la tarjeta de crédito, esto representa el 68.6% y el 31.4% respectivamente.

De acuerdo con el pronóstico realizado por el modelo, este alcanzó una exactitud superior al 89%, valor que estuvo en un punto medio entre la sensibilidad (73%) - que mide el comportamiento del modelo en la clasificación de los usuarios de tarjeta de crédito - y la especificidad (97.4%) - que mide el comportamiento del modelo frente a la clasificación de los no usuarios de tarjeta de crédito -.

Se destacan 3 nodos puros:

* "[NoTC:108;SiTC:0]" en donde los clientes deben cumplir con la regla: Si TravelSpend>1567.875 AND Income>59098.0
* "[NoTC:0;SiTC:47]" en donde los clientes deben cumplir con la regla: TravelSpend<=1188 AND FoodSpend>4990.73
* "[NoTC:1;SiTC:0]" en donde los clientes deben cumplir con la regla: TravelSpend<=1188 AND FoodSpend<=4990.73 AND FoodSpend>4972.55

Con respecto al pronóstico de los datos, se puede observar un total de 2 individuos sin clasificación, de los cuales ninguno usará tarjeta de crédito para el pago de los gastos de viaje y de los alimentos en sus planes de viajar el próximo año. El individuo 1 hace parte de la regla TravelSpend>1567.875 AND Income>59098.0, la cual lleva al nodo puro "[NoTC:108;SiTC:0]". El individuo 2 hace parte de la regla TravelSpend>1567.875 AND Income<=59098.0 AND FoodSpend>2711.895 AND TravelSpend<=3008.25, la cual lleva al nodo "[NoTC:107;SiTC:29]".